In [32]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from env import user, password, host
import warnings
warnings.filterwarnings('ignore')

def acquire_zillow():
    if os.path.exists('zillow_2017.csv'):
        return pd.read_csv('zillow_2017.csv', index_col=0)
    else:
        ''' Acquire data from Zillow using env imports and rename columns'''

        url = f"mysql+pymysql://{user}:{password}@{host}/zillow"

        query = """
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017
        LEFT JOIN propertylandusetype USING(propertylandusetypeid)
        WHERE propertylandusedesc IN ("Single Family Residential",                       
                                      "Inferred Single Family Residential")"""

        df = pd.read_sql(query, url)


        df = df.rename(columns = {'bedroomcnt':'bedrooms', 
                                  'bathroomcnt':'bathrooms', 
                                  'calculatedfinishedsquarefeet':'area',
                                  'taxvaluedollarcnt':'tax_value', 
                                  'yearbuilt':'year_built',})
        return df

In [42]:
zillow = acquire_zillow().reset_index()

In [43]:
zillow

,bedrooms,bathrooms,square_feet,tax_value,year_built,taxamount,fips
0,0.00,0.00,NaN,"27,516.00",NaN,NaN,"6,037.00"
1,0.00,0.00,NaN,10.00,NaN,NaN,"6,037.00"
2,0.00,0.00,NaN,10.00,NaN,NaN,"6,037.00"
3,0.00,0.00,NaN,"2,108.00",NaN,174.21,"6,037.00"
4,4.00,2.00,"3,633.00","296,425.00","2,005.00","6,941.39","6,037.00"
...,...,...,...,...,...,...,...
2152859,4.00,3.00,"2,262.00","960,756.00","2,015.00","13,494.52","6,059.00"
2152860,4.00,4.50,"3,127.00","536,061.00","2,014.00","6,244.16","6,059.00"
2152861,0.00,0.00,NaN,"208,057.00",NaN,"5,783.88","6,059.00"
2152862,3.00,2.50,"1,974.00","424,353.00","2,015.00","5,302.70","6,059.00"


In [44]:
type(zillow)

pandas.core.frame.DataFrame

In [45]:
zillow.shape

(2152864, 7)

In [46]:
zillow.describe()

,bedrooms,bathrooms,square_feet,tax_value,year_built,taxamount,fips
count,"2,152,853.00","2,152,853.00","2,144,379.00","2,152,371.00","2,143,526.00","2,148,422.00","2,152,864.00"
mean,3.29,2.23,"1,862.86","461,896.05","1,960.95","5,634.86","6,048.38"
std,0.95,1.00,"1,222.13","699,675.94",22.16,"8,178.91",20.43
min,0.00,0.00,1.00,1.00,"1,801.00",1.85,"6,037.00"
25%,3.00,2.00,"1,257.00","188,170.00","1,949.00","2,534.97","6,037.00"
50%,3.00,2.00,"1,623.00","327,671.00","1,958.00","4,108.94","6,037.00"
75%,4.00,3.00,"2,208.00","534,527.00","1,976.00","6,414.32","6,059.00"
max,25.00,32.00,"952,576.00","98,428,909.00","2,016.00","1,337,755.86","6,111.00"


In [57]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [70]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .75):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [71]:
handle_missing_values(zillow)

,bedrooms,bathrooms,square_feet,tax_value,year_built,taxamount,fips
3,0.00,0.00,NaN,"2,108.00",NaN,174.21,"6,037.00"
4,4.00,2.00,"3,633.00","296,425.00","2,005.00","6,941.39","6,037.00"
6,3.00,4.00,"1,620.00","847,770.00","2,011.00","10,244.94","6,037.00"
7,3.00,2.00,"2,077.00","646,760.00","1,926.00","7,924.68","6,037.00"
8,0.00,0.00,NaN,"6,730,242.00",NaN,"80,348.13","6,037.00"
...,...,...,...,...,...,...,...
2152859,4.00,3.00,"2,262.00","960,756.00","2,015.00","13,494.52","6,059.00"
2152860,4.00,4.50,"3,127.00","536,061.00","2,014.00","6,244.16","6,059.00"
2152861,0.00,0.00,NaN,"208,057.00",NaN,"5,783.88","6,059.00"
2152862,3.00,2.50,"1,974.00","424,353.00","2,015.00","5,302.70","6,059.00"


In [72]:
def data_prep(df, cols_to_remove=[], prop_required_column=.5, prop_required_row=.75):
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [79]:
data_prep(df)

,bedrooms,bathrooms,square_feet,tax_value,year_built,taxamount,fips
3,0.00,0.00,NaN,"2,108.00",NaN,174.21,"6,037.00"
4,4.00,2.00,"3,633.00","296,425.00","2,005.00","6,941.39","6,037.00"
6,3.00,4.00,"1,620.00","847,770.00","2,011.00","10,244.94","6,037.00"
7,3.00,2.00,"2,077.00","646,760.00","1,926.00","7,924.68","6,037.00"
8,0.00,0.00,NaN,"6,730,242.00",NaN,"80,348.13","6,037.00"
...,...,...,...,...,...,...,...
2152859,4.00,3.00,"2,262.00","960,756.00","2,015.00","13,494.52","6,059.00"
2152860,4.00,4.50,"3,127.00","536,061.00","2,014.00","6,244.16","6,059.00"
2152861,0.00,0.00,NaN,"208,057.00",NaN,"5,783.88","6,059.00"
2152862,3.00,2.50,"1,974.00","424,353.00","2,015.00","5,302.70","6,059.00"
